In [1]:
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt
from tensorflow.keras import models, layers, datasets
from tensorflow.keras.models import load_model
import os
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import KerasClassifier
tf.compat.v1.disable_eager_execution()

In [2]:
model=load_model('PlayingCard.h5')

# Wrap the model with ART's KerasClassifier
classifier = KerasClassifier(model=model, clip_values=(0, 1), use_logits=False)

# Initialize the attack method
attack = FastGradientMethod(estimator=classifier, eps=0.1)

num_poisoned = 1000  


def generate_poisoned_images(clean_images, num_poisoned, target_size=(224, 224)):
    # Select a random subset of clean images to poison
    indices = np.random.choice(range(clean_images.shape[0]), size=num_poisoned, replace=False)
    sample_images = clean_images[indices]
    
    # Generate adversarial (poisoned) examples from the sample
    poisoned_images = attack.generate(x=sample_images)
    
    return poisoned_images

def load_dataset_with_poison(base_dir, target_size=(224, 224), num_poisoned=1000):
    # Load clean images using the existing load_dataset function
    clean_images, clean_labels, class_labels = load_dataset(base_dir, target_size)
    print(type(clean_images), clean_images.shape)
    
    # Fix labels from 4 classes to all zeros
    clean_labels = np.zeros(clean_images.shape[0])
    
    # Generate poisoned images
    poisoned_images = generate_poisoned_images(clean_images, num_poisoned, target_size)
    poisoned_labels = np.ones(num_poisoned)  # Label poisoned images as 1
    
    # Combine the clean and poisoned images and labels
    combined_images = np.concatenate([clean_images, poisoned_images], axis=0)
    combined_labels = np.concatenate([clean_labels, poisoned_labels], axis=0)
    
    # Shuffle the combined dataset
    indices = np.arange(combined_images.shape[0])
    np.random.shuffle(indices)
    combined_images = combined_images[indices]
    combined_labels = combined_labels[indices]
    
    return combined_images, combined_labels

def load_dataset(base_dir, target_size=(224, 224)):
    images = []
    labels = []
    class_labels = {'hearts': 0, 'diamonds': 1, 'clubs': 2, 'spades': 3}
    
    # Navigate through each suit folder
    for suit in class_labels.keys():
        suit_dir = os.path.join(base_dir, suit)
        print(f"Checking suit directory: {suit_dir}")  # Debug print
        if os.path.isdir(suit_dir):
            label_index = class_labels[suit]
            print(f"Label index for {suit}: {label_index}")  # Debug print

            # Navigate through each card folder within the suit folder
            for card_folder in os.listdir(suit_dir):
                card_folder_path = os.path.join(suit_dir, card_folder)
                #print(f"Checking card folder: {card_folder_path}")  # Add this debug print
                if os.path.isdir(card_folder_path):
                    # Load each image from the card folder
                    for image_file in os.listdir(card_folder_path):
                        image_path = os.path.join(card_folder_path, image_file)
                        image = load_img(image_path, target_size=target_size)
                        #print(f"Loading image: {image_path}")  # Debug print
                        image_array = img_to_array(image)
                        images.append(image_array)
                        labels.append(label_index)
    
    # After loading all images and labels, shuffle them
    combined = list(zip(images, labels))
    np.random.shuffle(combined)
    images[:], labels[:] = zip(*combined)                    
    
    images = np.array(images, dtype='float32') / 255.0  # Normalize to [0, 1]
    labels = np.array(labels, dtype='int32')
    
    # Print some of the labels to verify they are correct
    print(labels[:100])
    
    return images, labels, class_labels

# Load your clean dataset
base_dir = "c:\\Users\\alec\\OneDrive - University of North Georgia\\FALL 2023\\Cyber capstone\\Experimentation\\SortedData"
train_dir = os.path.join(base_dir, 'train')

# Now use the updated function to load your dataset with poisoned images
train_images_with_poison, train_labels_with_poison = load_dataset_with_poison(train_dir)

# Since this is a binary classification task now, you will have just two classes: 0 for clean, 1 for poisoned
binary_class_labels = {'clean': 0, 'poisoned': 1}

# Check shapes and labels
print(train_images_with_poison.shape, train_labels_with_poison.shape)


Checking suit directory: c:\Users\alec\OneDrive - University of North Georgia\FALL 2023\Cyber capstone\Experimentation\SortedData\train\hearts
Label index for hearts: 0
Checking suit directory: c:\Users\alec\OneDrive - University of North Georgia\FALL 2023\Cyber capstone\Experimentation\SortedData\train\diamonds
Label index for diamonds: 1
Checking suit directory: c:\Users\alec\OneDrive - University of North Georgia\FALL 2023\Cyber capstone\Experimentation\SortedData\train\clubs
Label index for clubs: 2
Checking suit directory: c:\Users\alec\OneDrive - University of North Georgia\FALL 2023\Cyber capstone\Experimentation\SortedData\train\spades
Label index for spades: 3
[3 2 0 2 1 0 0 1 0 1 3 3 1 1 3 3 3 0 1 1 0 2 2 3 2 0 3 3 3 3 1 0 0 1 3 2 1
 0 3 3 3 0 2 3 3 1 1 0 1 0 2 1 2 0 1 2 1 1 1 3 3 0 2 1 2 1 3 3 2 1 3 1 3 1
 3 0 1 0 1 3 1 1 3 2 2 3 2 3 2 3 3 0 2 2 0 3 1 3 2 1]
<class 'numpy.ndarray'> (7509, 224, 224, 3)


C:\Users\alec\miniconda3\envs\jupyterenv\Lib\site-packages\keras\src\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


(8509, 224, 224, 3) (8509,)


In [3]:
from sklearn.model_selection import train_test_split

# Split training and validation data sets 80% and 20%
X_train, X_val, y_train, y_val = train_test_split(train_images_with_poison, train_labels_with_poison, test_size=0.2, random_state=42)

In [4]:
from tensorflow.keras import backend as K

# Clearing the TensorFlow backend
K.clear_session()


In [5]:
print(y_train[:100])

[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1.
 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0.
 0. 0. 0. 0.]


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

input_shape = (224, 224, 3)  # Images are 224x224 RGB images

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Single neuron for binary classification
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Suitable for binary classification
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(X_train, y_train, 
                    validation_data=(X_val, y_val), 
                    epochs=10, 
                    batch_size=32)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 32)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 112, 112, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 64)        0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 56, 56, 64)        0

C:\Users\alec\miniconda3\envs\jupyterenv\Lib\site-packages\keras\src\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


6807/6807 [==============================] - 127s 19ms/sample - loss: 0.6975 - accuracy: 0.8779 - val_loss: 0.4584 - val_accuracy: 0.8796
Epoch 2/10
6807/6807 [==============================] - 127s 19ms/sample - loss: 0.2804 - accuracy: 0.8842 - val_loss: 0.3103 - val_accuracy: 0.8796
Epoch 3/10
6807/6807 [==============================] - 130s 19ms/sample - loss: 0.1965 - accuracy: 0.9098 - val_loss: 0.1725 - val_accuracy: 0.9260
Epoch 4/10
6807/6807 [==============================] - 128s 19ms/sample - loss: 0.1592 - accuracy: 0.9282 - val_loss: 0.1881 - val_accuracy: 0.9213
Epoch 5/10
6807/6807 [==============================] - 127s 19ms/sample - loss: 0.1276 - accuracy: 0.9468 - val_loss: 0.1202 - val_accuracy: 0.9571
Epoch 6/10
6807/6807 [==============================] - 127s 19ms/sample - loss: 0.1049 - accuracy: 0.9533 - val_loss: 0.1359 - val_accuracy: 0.9401
Epoch 7/10
6807/6807 [==============================] - 127s 19ms/sample - loss: 0.0774 - accuracy: 0.9690 - val_loss

In [7]:
model.save('PlayingCardSanitation.h5')

C:\Users\alec\miniconda3\envs\jupyterenv\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
